# Interpretar código con LLM

Este cuaderno muestra como utilizar python para escanear un directorio de código fuente y generar resúmenes en Markdown con un LLM compatible con la API de OpenAI. En este caso se utilizó `qwen2.5-coder-7b-instructqwen2.5-coder-7b-instruct` en LMStudio. Estos resúmenes, combinados con una estrategia de RAG, podrían permitir navegar proyectos grandes, y superar la limitación del tamaño de la ventana de contexto.

In [9]:
SRC_PATH="../../../../apps/calcjob"
!tree -I build {SRC_PATH}

../../../../apps/calcjob
├── CalcJob.hpp
├── CMakeLists.txt
├── main.cpp
├── Manager.hpp
├── README.md
├── Reporter.hpp
└── Worker.hpp

0 directories, 7 files


## 1. Escanear el directorio

In [64]:
from pathlib import Path

class SourceCodeScanner:
    def __init__(self, source_dirs, extensions, exclude_dirs, relative_paths=True):
        """
        relative_paths=True → las rutas de los valores se guardan relativas al source_dir.
        """
        self.source_dirs = source_dirs
        self.extensions = extensions
        self.exclude_dirs = exclude_dirs
        self.relative_paths = relative_paths

    def scan(self):
        self.files = {}
        for source_dir in self.source_dirs:
            for file_path in source_dir.rglob("*"):
                if not file_path.is_file():
                    continue
                if any(part in self.exclude_dirs for part in file_path.parts):
                    continue
                if file_path.suffix in self.extensions:
                    key = file_path.relative_to(source_dir)
                    if self.relative_paths:
                        value = str(key)
                    else:
                        value = str(file_path.resolve())
                    self.files[str(key)] = value

    def get_files(self):
        """
        Devuelve un iterable (key, value) → (ruta_relativa, ruta_relativa/absoluta)
        """
        return self.files.items()

In [65]:
sc = SourceCodeScanner(
    source_dirs = [Path(SRC_PATH)], 
    extensions = [".cpp", ".h", ".hpp", "CMakeLists.txt"],  # extensiones que quieres incluir
    exclude_dirs = ["build"]
)
sc.scan()

sc.get_files()

dict_items([('Worker.hpp', 'Worker.hpp'), ('Reporter.hpp', 'Reporter.hpp'), ('Manager.hpp', 'Manager.hpp'), ('main.cpp', 'main.cpp'), ('CalcJob.hpp', 'CalcJob.hpp')])

In [66]:
from openai import OpenAI
import re  

class SourceCodeInterpreter:
    LLM_MODEL_NAME = "qwen2.5-coder-7b-instruct"

    def __init__(self):
        self.client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
        self.system_prompt = (
            "Eres un analista de código. "
            "Lee el siguiente código fuente y proporciona un **resumen conciso** en **Markdown**, "
            "en **español**, que incluya:\n\n"
            "- Una breve descripción del propósito del archivo.\n"
            "- Relación con otros archivos, o uso de clases o funciones de otros módulos.\n"
            "- Una lista de viñetas con las clases y funciones principales con una nota breve.\n"
            "- Cualquier detalle de diseño importante.\n"
            "- Mantén el formato claro, profesional y bien estructurado.\n"
            "- Comienza el reporte con una cabecera de segundo nivel (##).\n"
        )

    def query(self, file_content):
        completion = self.client.chat.completions.create(
            model=self.LLM_MODEL_NAME,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": file_content},
            ],
            temperature=0.0,
        )
        return completion.choices[0].message.content

def read_file(file_path):
    with open(file_path, "r") as f:
        return f.read()
    
def extract_markdown(text):
    """
    Extrae el contenido dentro de ```markdown ... ```
    Si no se encuentra, devuelve el texto tal cual.
    """
    pattern = r"```(?:markdown)?\n(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

In [68]:
from IPython.display import Markdown, display

sci = SourceCodeInterpreter()

for filename,relative_path in sc.get_files():    
    display(Markdown(f"# Archivo: {filename}"))
    display(Markdown("---"))
    raw_response = sci.query(read_file(Path(SRC_PATH) / Path(relative_path)))
    markdown_text = extract_markdown(raw_response)
    display(Markdown(markdown_text))
    display(Markdown("---"))

# Archivo: Worker.hpp

---

## Resumen del Código

Este archivo define una clase `Worker` que se encarga de procesar tareas asincrónicas utilizando un hilo separado. La clase interactúa con otras clases y módulos a través de la inclusión de `CalcJob.hpp`, lo que sugiere que `Worker` trabaja con objetos de tipo `CalcJob`.

### Relación con Otros Archivos
- **CalcJob.hpp**: Este archivo probablemente contiene la definición de la clase `CalcJob`, que representa una tarea o trabajo a procesar.

### Clases y Funciones Principales

- **Worker**:
  - **Callbacks**: La clase utiliza un tipo de función (`Callback`) para manejar el evento cuando una tarea se completa.
  
- **Métodos Publicos**:
  - `start()`: Inicia el hilo de trabajo.
  - `stopWorker()`: Detiene el hilo de trabajo y espera a que termine.
  - `enqueueJob(const CalcJob& job)`: Encola una nueva tarea para procesar.
  - `onJobDone(Callback cb)`: Establece un callback que se ejecuta cuando una tarea se completa.

- **Métodos Privados**:
  - `run()`: Método principal del hilo de trabajo, donde se espera y procesa las tareas.
  - `process(CalcJob& job)`: Procesa la tarea según el operador especificado en `CalcJob`.

### Detalles de Diseño Importantes

- **Concurrencia**: Utiliza un hilo separado para procesar tareas, lo que permite una ejecución concurrente.
- **Sincronización**: Se utiliza un mutex (`mtx`) y una variable condicional (`cv`) para sincronizar el acceso a la cola de tareas entre el hilo principal y el hilo de trabajo.
- **Seguridad de Cierre**: El método `stopWorker()` asegura que el hilo termina correctamente incluso si no hay más tareas en la cola.

Este diseño permite una implementación eficiente y segura para procesar tareas asincrónicas, con un buen manejo de concurrencia y sincronización.

---

# Archivo: Reporter.hpp

...

---

# Archivo: CalcJob.hpp

---

## Descripción del Archivo

El archivo proporciona una definición de estructura para un trabajo de cálculo (`CalcJob`). Esta estructura se utiliza para encapsular los detalles necesarios para realizar operaciones matemáticas básicas entre dos números.

## Relación con Otros Archivos o Uso de Clases/Funciones de Otros Módulos

Esta estructura no depende explícitamente de otros archivos o clases. Se puede utilizar en cualquier módulo que requiera realizar cálculos basados en operaciones aritméticas entre dos números.

## Clases y Funciones Principales

- **`CalcJob`**: Estructura que encapsula los detalles necesarios para un trabajo de cálculo.
  - `a`, `b`: Dos valores flotantes que representan los operandos del cálculo.
  - `op`: Enumeración que define la operación a realizar (`Add`, `Sub`, `Mul`, `Div`).
  - `result`: Valor flotante que almacena el resultado del cálculo.

## Detalles de Diseño Importantes

- La estructura `CalcJob` es simple y directa, lo que facilita su uso en diferentes partes del código.
- El uso de una enumeración para las operaciones permite un manejo claro y seguro de los tipos de operaciones permitidas.
- La inclusión de un miembro `result` preinicializado a 0.0f asegura que siempre haya un valor válido para el resultado, incluso si no se ha realizado ninguna operación.

Este diseño es adecuado para aplicaciones donde se necesitan realizar cálculos básicos y los resultados deben ser almacenados de manera eficiente.

---